In [1]:
# Imports
import numpy as np
import pandas as pd
import math as mt
import matplotlib.pyplot as plt
from scipy.stats import linregress as lr
from PIL import Image
from typing import List

In [2]:
# ArcticDEM data
Image.MAX_IMAGE_PIXELS = None
file_name = "datasets/2012/SETSM_s2s041_WV02_20120522_1030010018922400_10300100188CAD00_2m_lsf_seg1_dem.tif"
image = Image.open(file_name)

# Create mask
raster = np.array(image) # 1D array of the image
mask = ~(raster == -9999) # 1D array of T/F - False if data missing

# Image Data
shape = raster.shape
dimension = raster.ndim
average = np.mean(raster[mask])
low = np.min(raster[mask])
high = np.max(raster[mask])

# Print Image Data
arctic_dataframe = pd.DataFrame({"Image Shape": [shape], "Image Dimension": [dimension], "Average Elevation": [average], "Minimum Elevation": [low], "Maximum Elevation": [high]})
print(arctic_dataframe)

# Image File is too large - MemoryError
# Show Image
# plt.imshow(raster, cmap='Accent')
# plt.colorbar(label='Elevation')
# plt.title('Elevation Data')
# plt.xlabel('X coord (2m)')
# plt.ylabel('Y coord (2m)')
# plt.show()

# DEM strip and mosaic ﬁles are distributed at a ground sample distance (GSD) of 2 meters


      Image Shape  Image Dimension  Average Elevation  Minimum Elevation  \
0  (55704, 28243)                2         854.531555         300.390625   

   Maximum Elevation  
0        1780.367188  


In [9]:
# Box Counting
def box_count(raster: np.ndarray, mask: np.ndarray, box_size: List[int]):
    rows, cols = raster.shape
    total_box_count = 0 
    for size in box_size:
        box_count_for_size = 0
        for i in range(0, rows, size):
            for j in range(0, cols, size):
                end_row = min(i + size, rows)
                end_col = min(j + size, cols)

                current_slice = raster[i:end_row, j:end_col]
                current_mask = mask[i:end_row, j:end_col]

                valid_vals = current_slice[current_mask]
                if valid_vals.size > 0:
                    max_val = np.max(valid_vals)
                    min_val = np.min(valid_vals)
                    hdiff = max_val - min_val
                    box_count = mt.ceil(hdiff / size)
                    box_count_for_size += box_count
                    total_box_count += box_count
        print(f"Total box count for box size {size} is {box_count_for_size}")
    return total_box_count



In [10]:
box_sizes = [2, 4, 8, 16, 32, 64, 128, 256]
box_count(raster, mask, box_sizes)

Total box count for box size 2 is 126243397
Total box count for box size 4 is 31994176
Total box count for box size 8 is 8053668
Total box count for box size 16 is 2019944
Total box count for box size 32 is 506035
Total box count for box size 64 is 126817
Total box count for box size 128 is 31939
Total box count for box size 256 is 8116


168984092